## Data set preparation for ML model development - PCA based variable pool with RTMA data

### Set up/check environment

In [ ]:
# Check environment
!conda info

In [ ]:
# Import packages 
import pandas as pd
import random
import numpy as np
import sklearn
from sklearn import datasets
from datetime import datetime
from itertools import cycle
import glob2
import os
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import LabelBinarizer as lb

# set the number of maximum displayed rows for printed dataframes to 1000
pd.set_option('display.max_rows', 1000)

In [ ]:
# plot raw data

PCA_df = pd.read_csv('')

# including 'row_shading instead of canopy closure and row spacing'
row_shading = PCA_df['canopy_avg'] * PCA_df['spacing (m)']
insert_index = PCA_df.columns.get_loc('canopy_avg') + 1  # Insert after col1
PCA_df.insert(insert_index, 'row_shading', row_shading)

sns.set_theme()
sns.set(font_scale=0.5) 
ax = sns.catplot(data=PCA_df, kind = 'bar')
ax.set_xticklabels(rotation=90, ha="right")
ax.set(title = 'Raw data')

#### Max absolute rescaling

In [ ]:
# copy the data - only the numerical data
dt_max_scaled = PCA_df.iloc[:, 7:]

# apply normalization techniques
for column in dt_max_scaled.columns:
    dt_max_scaled[column] = dt_max_scaled[column]  / dt_max_scaled[column].abs().max()
    
# plot normalized data
sns.set_theme()
sns.set(font_scale=0.5) 
ax = sns.catplot(data=dt_max_scaled, kind = 'bar')
ax.set_xticklabels(rotation=90, ha="right")
ax.set(title = 'Max absolute rescaled data')

In [ ]:
# apply PCA - max absolute
pca = PCA(n_components=None) # no limit to number of PCAS
pca.fit(dt_max_scaled) # performing PCA

# retrieve the eigenvalues
#print("Eigenvalues:")
#print(pca.explained_variance_)
#print()

# return explained variances
#print("Variances (Percentage):")
#print(pca.explained_variance_ratio_ * 100)
#print()

# plot the scree plot
plt.figure(figsize=(3.25, 3.25), dpi=1200)
plt.plot(np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel("Number of components (Dimensions)")
plt.ylabel("Explained variance (%)")
plt.title('Max absolute rescaling - RTMA\nNumber of PCs needed to explain 90% of variance: 5')

# return the number of PCs needed to explain 95% of the variance
pca = PCA(0.90) # set threshold for explained variance to 90%
principalComponents = pca.fit_transform(dt_max_scaled) # perform PCA
print("number of PCs needed to explain 90% of variance:",  np.shape(principalComponents)[1])

### Perform PCA to achieve 0.9 explained variance

In [ ]:
# dictating thresold for number of PCs
pca = PCA(0.90) 
# calculate principal components - using max absolute rescaled values
principalComponents = pca.fit_transform(dt_max_scaled) 
# creating column names for principal components
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 
                                                                  'principal component 3', 'principal component 4',
                                                                  'principal component 5'])

### Merge PCA data and target data for training and testing data sets

In [ ]:
# merging principal components with targets (binary classification for each data point)
# changing the array of solutions to a single column data frame
dt_sol = pd.DataFrame(PCA_df.target) 

# merging predictor data (principal components and withheld categorical predictors) with the solutions   
pc_df = pd.merge(dt_sol, principalDf, left_index=True, right_index=True)

### Data splitting (original - no stratification)
* Training: 80%, testing 20%  
* Stratification by apothecia threshold binary

In [ ]:
# split into x and y
xdat = pd.merge(PCA_df.iloc[:, 3], pc_df.loc[:, pc_df.columns != 'target' ], left_index = True, right_index = True) 
ydat = pd.DataFrame(pc_df.loc[:, 'target'])

# using binary encoding to transform categorical soil type
xdat['soil type'].replace(['sand', 'loamy sand', 'loam'], [0, 1, 2], inplace=True)

In [ ]:
# stratifying by dt_sol
x_train, x_test, y_train, y_test = train_test_split(xdat, ydat, test_size=0.20, random_state=42, stratify=dt_sol)

In [ ]:
# checking that indexes for data split between x and y match for training and test
print('x training indexes:', x_train.index)
print('y training indexes:', y_train.index)

print('x testing indexes:', x_test.index)
print('y testing indexes:', y_test.index)

In [ ]:
# checking stratification (equal proportions of positive solutions (target = 1) between train and test)
print(sum(y_train['target'])/len(y_train))
print(sum(y_test['target'])/len(y_test))

### PCA - data load out

In [ ]:
x_train.to_csv('', index=False, header=True)
y_train.to_csv('', index=False, header=True)
x_test.to_csv('', index=False, header=True)
y_test.to_csv('', index=False, header=True)